In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Tree II').getOrCreate()

In [2]:
df = spark.read.csv('/home/ubuntu/Course_Notes/Spark_for_Machine_Learning/Tree_Methods/College.csv',inferSchema=True, header=True)

In [5]:
df.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [6]:
df.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
df.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [10]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [11]:
output = assembler.transform(df)

In [12]:
from pyspark.ml.feature import StringIndexer

In [13]:
indexer = StringIndexer(inputCol= 'Private',outputCol='private_indexer')

In [14]:
output_fixed = indexer.fit(output).transform(output)

In [15]:
output_fixed.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- features: vector (nullable = true)
 |-- private_indexer: double (nullable = true)



In [16]:
final_data = output_fixed.select(['features','private_indexer'])

In [17]:
final_data.show()

+--------------------+---------------+
|            features|private_indexer|
+--------------------+---------------+
|[1660.0,1232.0,72...|            0.0|
|[2186.0,1924.0,51...|            0.0|
|[1428.0,1097.0,33...|            0.0|
|[417.0,349.0,137....|            0.0|
|[193.0,146.0,55.0...|            0.0|
|[587.0,479.0,158....|            0.0|
|[353.0,340.0,103....|            0.0|
|[1899.0,1720.0,48...|            0.0|
|[1038.0,839.0,227...|            0.0|
|[582.0,498.0,172....|            0.0|
|[1732.0,1425.0,47...|            0.0|
|[2652.0,1900.0,48...|            0.0|
|[1179.0,780.0,290...|            0.0|
|[1267.0,1080.0,38...|            0.0|
|[494.0,313.0,157....|            0.0|
|[1420.0,1093.0,22...|            0.0|
|[4302.0,992.0,418...|            0.0|
|[1216.0,908.0,423...|            0.0|
|[1130.0,704.0,322...|            0.0|
|[3540.0,2001.0,10...|            1.0|
+--------------------+---------------+
only showing top 20 rows



In [18]:
train,test = final_data.randomSplit([0.7,0.3])

In [19]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

In [22]:
dtc = DecisionTreeClassifier(labelCol='private_indexer')
gbc = GBTClassifier(labelCol='private_indexer')
rfc = RandomForestClassifier(labelCol='private_indexer')

In [23]:
dtc_model = dtc.fit(train)
gbc_model = gbc.fit(train)
rfc_model = rfc.fit(train)

In [24]:
dtc_preds = dtc_model.transform(test)
gbc_preds = gbc_model.transform(test)
rfc_preds = rfc_model.transform(test)

In [25]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
evaluator = MulticlassClassificationEvaluator(metricName='accuracy',labelCol='private_indexer')

In [31]:
print(evaluator.evaluate(dtc_preds))
print(evaluator.evaluate(gbc_preds))
print(evaluator.evaluate(rfc_preds))

0.9219512195121952
0.926829268292683
0.9512195121951219
